In [ ]:
import pandas as pd

In [ ]:
df_wahl = pd.read_csv(r'C:\Users\DE88342\OneDrive - Grunenthal Group\Desktop\VSC - Py\DPArenamed\Data\IMPORT_BOT2_Wahl.csv', delimiter= ';')
df_event = pd.read_csv(r'C:\Users\DE88342\OneDrive - Grunenthal Group\Desktop\VSC - Py\DPArenamed\Data\IMPORT_BOT1_Veranstaltungsliste.csv', delimiter= ';')
df_raum = pd.read_csv(r'C:\Users\DE88342\OneDrive - Grunenthal Group\Desktop\VSC - Py\DPArenamed\Data\IMPORT_BOT0_Raumliste.csv', delimiter= ';')

In [ ]:
df_wahl.head(20)

In [ ]:
value_counts = pd.concat([df_wahl['Wahl 1'].value_counts(),
                         df_wahl['Wahl 2'].value_counts(),
                         df_wahl['Wahl 3'].value_counts(),
                         df_wahl['Wahl 4'].value_counts(),
                         df_wahl['Wahl 5'].value_counts(),
                         df_wahl['Wahl 6'].value_counts()], axis=1)

column_names = ['Wahl1', 'Wahl2', 'Wahl3', 'Wahl4', 'Wahl5', 'Wahl6']
value_counts.columns = column_names

value_counts.insert(0, 'event_number', value_counts.index)

print(value_counts)


In [ ]:
df_event = df_event.rename(columns={'Nr. ': 'event_number'})
df_event

In [ ]:
def count_company_votes(df_wahl, df_event):
    
    # Sum the votes for each company
    company_votes = value_counts.sum(axis=1)
    
    # Add the votes for companies with different indices
    company_votes['Finanzamt'] = value_counts['Wahl1'].get(18, 0) + value_counts['Wahl1'].get(19, 0)
    
    # Divide the total votes by 20 to get the needed slots
    needed_slots = company_votes.sum() / 20
    
    return needed_slots

# Call the function with the loaded DataFrames
needed_slots = count_company_votes(df_wahl, df_event)


In [ ]:
import math
merged_df = pd.merge(df_event, value_counts, on='event_number')

merged_df['Total 1-3'] = merged_df[['Wahl1', 'Wahl2', 'Wahl3']].sum(axis=1).astype(int)

merged_df['Total 4-6'] = merged_df[['Wahl4', 'Wahl5', 'Wahl6']].sum(axis=1).astype(int)

merged_df['Total'] = merged_df['Total 1-3'] + merged_df['Total 4-6']

merged_df['Nötige Slots'] = merged_df['Total 1-3'] / merged_df['Max. Teilnehmer']

merged_df['Optionale Slots'] = merged_df['Total 4-6'] / merged_df['Max. Teilnehmer']

# Round the 'Nötige Slots' column up or down
merged_df['Rundet Nötige Slots'] = merged_df.apply(lambda row: math.ceil(row['Nötige Slots']) if row['Nötige Slots'] - math.floor(row['Nötige Slots']) >= (5 / row['Max. Teilnehmer']) else math.floor(row['Nötige Slots']), axis=1)

# Round the 'Optionale Slots' column up or down
merged_df['Rundet Optionale Slots'] = merged_df.apply(lambda row: math.ceil(row['Optionale Slots']) if row['Optionale Slots'] - math.floor(row['Optionale Slots']) >= (5 / row['Max. Teilnehmer']) else math.floor(row['Optionale Slots']), axis=1)

merged_df

In [ ]:
df_slots = pd.DataFrame(columns=['A', 'B', 'C', 'D', 'E'])

df_timetable = pd.concat([df_raum, df_slots[['A', 'B', 'C', 'D', 'E']]], axis=1)

In [ ]:
# Sort df_merged by 'Total 1-3' in descending order
df_merged_sorted = merged_df.sort_values('Total 1-3', ascending=False)

# Iterate over the sorted events
for _, event in df_merged_sorted.iterrows():
    event_number = event['event_number']
    rundet_slots = event['Rundet Nötige Slots']
    
    # Find the room with the least number of events already assigned
    min_events_room = df_timetable.iloc[:, 1:].apply(lambda row: row.notnull().sum(), axis=1).idxmin()
    
    # Assign the event to the room with the least number of events
    room_row = df_timetable.loc[min_events_room]
    available_slots = rundet_slots - room_row[1:].eq(event_number).sum()
    available_columns = room_row[1:][room_row[1:].isnull()].index.tolist()
    for i in range(min(available_slots, len(available_columns))):
        df_timetable.loc[min_events_room, available_columns[i]] = event_number

#TODO: check for frühester startzeitpunkt
        
#TODO: check for same company in ONE room 
        
#TODO: 

df_timetable